In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
from langchain_core.prompts import PromptTemplate

question = '인프런에는 어떤 강의가 있나요?'
prompt_with_template = f"아래 질문에 답변해주세요: \n\n {question}"

prompt_template = PromptTemplate(template=prompt_with_template, input_variables=["question"])

하드 코딩

In [4]:
llm.invoke(prompt_template.invoke({"question": question}))

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등 다양한 기술과 주제에 대한 강의를 찾을 수 있습니다. 각 강의는 강사에 따라 내용과 수준이 다르므로, 자신의 필요에 맞는 강의를 선택할 수 있습니다. 인프런 웹사이트를 방문하면 최신 강의 목록과 상세 정보를 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 27, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxI3WnzieVyRlZiThL0cM4LCsQER', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f6823a1d-4b9d-48f8-9c54-7e37a5e858a0-0', usage_metadata={'input_tokens': 27, 'output_tokens': 130, 'total_tokens': 157, 'input_token_details': {'audio': 0

체인 사용

In [5]:
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question": question})

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅, 그리고 프로젝트 관리와 같은 주제의 강의를 찾을 수 있습니다. 각 강의는 강사에 따라 다르며, 초급부터 고급 과정까지 다양한 수준의 강의가 마련되어 있습니다. 특정 강의에 대한 정보는 인프런 웹사이트를 방문하여 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 27, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxIAYDOXAAXFF6ANqBb80Gyc6NYo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e8e023cf-ec1f-4288-aee8-72f79bd86190-0', usage_metadata={'input_tokens': 27, 'output_tokens': 132, 'total_tokens': 159, 'input_token_details': {'audio': 

# WebBaseLoader를 사용한 RAG

In [7]:
%pip install -qU beatifulsoup4

ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4
Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 전처리

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200  # 이전 청크와 겹치게 가져올 토큰 수. 유사도 검색을 할 때 원하는 문서를 가져올 수 있는 확률을 높히는 방식
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [10]:
document_list

[Document(metadata={'source': 'https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=', 'title': '인프런 - 라이프타임 커리어 플랫폼', 'description': '프로그래밍, 인공지능, 데이터, 마케팅, 디자인등 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 곳.', 'language': 'ko'}, page_content='인프런 - 라이프타임 커리어 플랫폼NN인프런 - 라이프타임 커리어 플랫폼')]

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name="inflearn-lectures",
    persist_directory="./inflearn-lectures"
)

## 2. 데이터 검색을 위한 Retriever 생성

In [15]:
retriever = database.as_retriever(search_kwargs={'k': 1})

## 3. 프롬프트 생성

1. 수동 생성

In [16]:
rag_prompt_template = PromptTemplate(
    template='''
    You are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
    ''',
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retriever, "question": question})

AIMessage(content='인프런에는 다양한 주제의 강의가 있습니다. 주로 프로그래밍, 데이터 과학, 인공지능, 웹 개발, 모바일 개발, 디자인 등 여러 분야의 강의를 제공하고 있습니다. 각 강의는 초급부터 고급까지 다양한 수준으로 구성되어 있으며, 실습 중심의 강의도 많아 학습에 도움이 됩니다. 특정 강의에 대한 정보가 필요하시다면, 인프런 웹사이트를 방문하여 검색해 보시는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 90, 'total_tokens': 193, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxZRa7SjF2g6ZCK0NqsmDYrN3rYW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f45660e7-ee33-4d67-a0e3-cf128c51d412-0', usage_metadata={'input_tokens': 90, 'output_tokens': 103, 'total_tokens': 193, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin

2. LangChain Hub 제공 프롬프트 활용

In [17]:
from langchain import hub

rlm_rag_prompt = hub.pull("rlm/rag-prompt")

rlm_rag_chain = rlm_rag_prompt | llm
rlm_rag_chain.invoke({"context": retriever, "question": question})

/Users/kimjaehee/Desktop/Study_file/langchain/rag_langchain/.venv/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


AIMessage(content='인프런에는 다양한 강의가 있습니다. 주로 프로그래밍, 데이터 과학, 디자인, 마케팅 등 여러 분야의 강의를 제공합니다. 구체적인 강의 내용은 인프런 웹사이트에서 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 114, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxa62Nwox7l8Q1BQm9je6MFHs5Vi', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e31a1cb9-d546-4a84-939e-7b1751016411-0', usage_metadata={'input_tokens': 114, 'output_tokens': 50, 'total_tokens': 164, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

3. `RunnablePassThrough` 활용

In [18]:
from langchain_core.runnables import RunnablePassthrough

rag_pass_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    rlm_rag_prompt |
    llm
)
rag_pass_chain.invoke(question)

AIMessage(content='인프런에는 프로그래밍, 인공지능, 데이터, 마케팅, 디자인 등 다양한 강의가 있습니다. 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 기회를 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 220, 'total_tokens': 265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYxeHiFI3YzhtHkabkYhu6ogGhddU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--47df5022-74af-4da5-b5e7-5555bb560f57-0', usage_metadata={'input_tokens': 220, 'output_tokens': 45, 'total_tokens': 265, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})